In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# 🍎 맥북(macOS) 전용 한글 폰트 설정
plt.rc('font', family='AppleGothic') 
# 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False 

# 확인용: 데이터 불러오기 준비
print("라이브러리 로드 완료!")

라이브러리 로드 완료!


In [6]:
"""
서울시 5대 범죄 데이터 전처리 - 머신러닝용
1. 시계열 데이터 재구조화 (2014-2024)
2. 5대 범죄 필터링 및 통합
3. 장소 카테고리 4대 분류
"""

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# ============================================
# 공통 함수: 인코딩 처리
# ============================================

def load_csv_with_encoding(file_path):
    """cp949 -> utf-8 순으로 인코딩 시도"""
    encodings = ['cp949', 'utf-8', 'euc-kr']
    
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"✅ 파일 로드 성공 (인코딩: {encoding})")
            return df
        except:
            continue
    raise ValueError(f"❌ 파일을 읽을 수 없습니다: {file_path}")


print("="*80)
print("📊 서울시 5대 범죄 데이터 전처리 - 머신러닝용")
print("="*80)


# ============================================
# PART 1: 시계열 데이터 재구조화 (5대 범죄 발생현황)
# ============================================

print("\n[PART 1] 시계열 데이터 재구조화 (2014-2024)")
print("-"*80)

# 1-1. 데이터 로드
df_yearly = load_csv_with_encoding('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/csv/5대+범죄+발생현황_20260122174801.csv')

print(f"\n원본 데이터 형태: {df_yearly.shape}")

# 1-2. 헤더 정보 추출
# 첫 3행이 헤더 정보를 담고 있음
header_row1 = df_yearly.iloc[0].tolist()  # 합계/소계 정보
header_row2 = df_yearly.iloc[1].tolist()  # 범죄 종류
header_row3 = df_yearly.iloc[2].tolist()  # 발생/검거 구분

# 실제 데이터는 3행부터 시작
df_data = df_yearly.iloc[3:].copy()
df_data.columns = df_yearly.columns

print(f"데이터 행 수: {len(df_data)}")

# 1-3. 연도별 컬럼 매핑 생성
# 컬럼명 패턴: 2014, 2014.1, 2014.2, ... → 각각 다른 범죄/발생-검거 조합
columns_info = []

for i, col in enumerate(df_yearly.columns):
    if i < 2:  # 자치구별(1), 자치구별(2) 스킵
        continue
    
    # 연도 추출
    if '.' in str(col):
        year = str(col).split('.')[0]
    else:
        year = str(col)
    
    # 범죄 종류와 발생/검거 정보
    crime_type = header_row2[i]
    data_type = header_row3[i]  # '발생' 또는 '검거'
    
    columns_info.append({
        'original_col': col,
        'year': year,
        'crime_type': crime_type,
        'data_type': data_type
    })

# DataFrame으로 변환하여 확인
col_info_df = pd.DataFrame(columns_info)
print(f"\n컬럼 정보 샘플:")
print(col_info_df.head(10))

# 1-4. '발생' 데이터만 필터링하여 Melt
# 5대 범죄 목록 (시계열 데이터에는 이미 폭력으로 통합되어 있음)
target_crimes = ['살인', '강도', '강간·강제추행', '절도', '폭력']

# 발생 데이터 컬럼만 선택
발생_columns = []
발생_years = []
발생_crimes = []

for info in columns_info:
    if info['data_type'] == '발생' and info['crime_type'] in target_crimes:
        발생_columns.append(info['original_col'])
        발생_years.append(info['year'])
        발생_crimes.append(info['crime_type'])

print(f"\n'발생' 데이터 컬럼 수: {len(발생_columns)}")

# 1-5. 데이터 추출 및 재구조화
# 자치구 컬럼 확인 (실제 데이터에서 자치구명은 두 번째 컬럼에 있음)
df_data_clean = df_data.reset_index(drop=True)

# 필요한 컬럼만 선택: 자치구별(2) + 발생 데이터 컬럼들
district_col = '자치구별(2)'
selected_cols = [district_col] + 발생_columns

df_selected = df_data_clean[selected_cols].copy()

# 자치구 이름 정제 ('합계'와 '소계' 제외)
df_selected = df_selected[~df_selected[district_col].isin(['소계', '합계'])].copy()
df_selected = df_selected.rename(columns={district_col: 'District'})

print(f"\n자치구 수: {len(df_selected)}")
print(f"자치구 목록: {df_selected['District'].tolist()}")

# 1-6. Melt 작업 수행
# 각 컬럼에 대해 연도와 범죄 종류 매핑
melted_data = []

for idx, row in df_selected.iterrows():
    district = row['District']
    
    for i, col in enumerate(발생_columns):
        year = 발생_years[i]
        crime = 발생_crimes[i]
        count = row[col]
        
        # 데이터 정제: 콤마 제거 및 결측치 처리
        if pd.isna(count) or count == '-' or count == '':
            count = 0
        else:
            count = str(count).replace(',', '')
            try:
                count = int(float(count))
            except:
                count = 0
        
        melted_data.append({
            'Year': int(year),
            'District': district,
            'Crime_Type': crime,
            'Count': count
        })

# DataFrame 생성
df_crime_yearly = pd.DataFrame(melted_data)

# 1-7. 성폭력 통일 (강간·강제추행 -> 성폭력)
df_crime_yearly['Crime_Type'] = df_crime_yearly['Crime_Type'].replace('강간·강제추행', '성폭력')

# 1-8. 정렬 및 최종 정리
df_crime_yearly = df_crime_yearly.sort_values(['Year', 'District', 'Crime_Type']).reset_index(drop=True)

print(f"✅ 시계열 데이터 재구조화 완료")
print(f"   - 최종 행 수: {len(df_crime_yearly):,}개")
print(f"   - 연도 범위: {df_crime_yearly['Year'].min()} ~ {df_crime_yearly['Year'].max()}")
print(f"   - 자치구 수: {df_crime_yearly['District'].nunique()}개")
print(f"   - 범죄 종류: {', '.join(df_crime_yearly['Crime_Type'].unique())}")


# ============================================
# PART 2: 범죄 발생 시간 데이터 전처리
# ============================================

print("\n\n[PART 2] 범죄 발생 시간 데이터 전처리")
print("-"*80)

# 2-1. 데이터 로드
df_time = load_csv_with_encoding('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/csv/범죄발생시간.csv')

# 2-2. 5대 범죄 필터링
crime_types = ['살인', '강도', '강간·강제추행', '성폭력', '절도', '폭행', '상해']
crime_col = '범죄분류'

df_time_filtered = df_time[df_time[crime_col].isin(crime_types)].copy()

print(f"필터링 결과: {len(df_time_filtered)}개 범죄 유형")

# 2-3. 시간대 컬럼 정제
time_columns = [col for col in df_time_filtered.columns if col != crime_col]

for col in time_columns:
    df_time_filtered[col] = df_time_filtered[col].replace('-', 0)
    df_time_filtered[col] = df_time_filtered[col].replace('', 0)
    df_time_filtered[col] = pd.to_numeric(df_time_filtered[col], errors='coerce').fillna(0).astype(int)

# 2-4. 폭력 = 폭행 + 상해
if '폭행' in df_time_filtered[crime_col].values and '상해' in df_time_filtered[crime_col].values:
    assault_row = df_time_filtered[df_time_filtered[crime_col] == '폭행'].iloc[0]
    injury_row = df_time_filtered[df_time_filtered[crime_col] == '상해'].iloc[0]
    
    violence_data = {crime_col: '폭력'}
    for col in time_columns:
        violence_data[col] = assault_row[col] + injury_row[col]
    
    df_time_filtered = df_time_filtered[~df_time_filtered[crime_col].isin(['폭행', '상해'])]
    df_time_filtered = pd.concat([df_time_filtered, pd.DataFrame([violence_data])], ignore_index=True)

# 2-5. 성폭력 통일
df_time_filtered[crime_col] = df_time_filtered[crime_col].replace('강간·강제추행', '성폭력')

df_time_final = df_time_filtered.reset_index(drop=True)

print(f"✅ 시간 데이터 전처리 완료")
print(f"   - 범죄 유형: {', '.join(df_time_final[crime_col].tolist())}")


# ============================================
# PART 3: 범죄 발생 장소 데이터 전처리
# ============================================

print("\n\n[PART 3] 범죄 발생 장소 데이터 전처리")
print("-"*80)

# 3-1. 데이터 로드
df_place = load_csv_with_encoding('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/csv/범죄발생장소.csv')

# 3-2. 5대 범죄 필터링
df_place_filtered = df_place[df_place[crime_col].isin(crime_types)].copy()

print(f"필터링 결과: {len(df_place_filtered)}개 범죄 유형")

# 3-3. 장소 카테고리 매핑
place_category_map = {
    # 주거지구
    '아파트_연립_다세대': '주거지구',
    '단독주택': '주거지구',
    
    # 업무지구
    '사무실': '업무지구',
    '공장': '업무지구',
    '학교': '업무지구',
    '부대': '업무지구',
    
    # 다중이용시설
    '노상': '다중이용시설',
    '상점': '다중이용시설',
    '시장노점': '다중이용시설',
    '숙박업소_목욕탕': '다중이용시설',
    '유흥접객업소': '다중이용시설',
    '역대합실': '다중이용시설',
    '지하철': '다중이용시설',
    '기타교통수단내': '다중이용시설',
    '흥행장': '다중이용시설',
    '유원지': '다중이용시설',
    '금융기관': '다중이용시설',
    '의료기관': '다중이용시설',
    '종교기관': '다중이용시설',
    
    # 기타
    '고속도로': '기타',
    '공사장_광산': '기타',
    '창고': '기타',
    '산야': '기타',
    '해상': '기타',
    '구금장소': '기타',
    '공지': '기타',
    '기타': '기타'
}

# 3-4. 카테고리별 데이터 통합
place_columns = [col for col in df_place_filtered.columns if col != crime_col]
category_data = {}

for place_col in place_columns:
    if place_col in place_category_map:
        category = place_category_map[place_col]
        
        temp_data = df_place_filtered[place_col].astype(str).replace('-', '0')
        temp_data = pd.to_numeric(temp_data, errors='coerce').fillna(0).astype(int)
        
        if category not in category_data:
            category_data[category] = temp_data
        else:
            category_data[category] = category_data[category] + temp_data

# 3-5. 카테고리별 DataFrame 생성
df_place_final = df_place_filtered[[crime_col]].copy().reset_index(drop=True)

for category in ['주거지구', '업무지구', '다중이용시설', '기타']:
    if category in category_data:
        df_place_final[category] = category_data[category].values

# 3-6. 폭력 = 폭행 + 상해
if '폭행' in df_place_final[crime_col].values and '상해' in df_place_final[crime_col].values:
    assault_row = df_place_final[df_place_final[crime_col] == '폭행'].iloc[0]
    injury_row = df_place_final[df_place_final[crime_col] == '상해'].iloc[0]
    
    violence_data = {crime_col: '폭력'}
    for category in ['주거지구', '업무지구', '다중이용시설', '기타']:
        if category in df_place_final.columns:
            violence_data[category] = assault_row[category] + injury_row[category]
    
    df_place_final = df_place_final[~df_place_final[crime_col].isin(['폭행', '상해'])]
    df_place_final = pd.concat([df_place_final, pd.DataFrame([violence_data])], ignore_index=True)

# 3-7. 성폭력 통일
df_place_final[crime_col] = df_place_final[crime_col].replace('강간·강제추행', '성폭력')

print(f"✅ 장소 데이터 전처리 완료")
print(f"   - 범죄 유형: {', '.join(df_place_final[crime_col].tolist())}")


# ============================================
# PART 4: 최종 결과 확인 및 저장
# ============================================

print("\n\n" + "="*80)
print("📋 최종 결과 확인")
print("="*80)

print("\n[1] df_crime_yearly - 시계열 데이터 (2014-2024)")
print("-"*80)
print(df_crime_yearly.head())
print(f"\n{df_crime_yearly.info()}")

print("\n\n[2] df_time_final - 시간대별 분포")
print("-"*80)
print(df_time_final.head())
print(f"\n{df_time_final.info()}")

print("\n\n[3] df_place_final - 장소 카테고리별 분포")
print("-"*80)
print(df_place_final.head())
print(f"\n{df_place_final.info()}")


# ============================================
# PART 5: 데이터 저장
# ============================================

print("\n\n" + "="*80)
print("💾 데이터 저장")
print("="*80)

output_dir = '/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/ouput_2'

df_crime_yearly.to_csv(output_dir + 'crime_yearly_2014_2024.csv', index=False, encoding='cp949')
df_time_final.to_csv(output_dir + 'crime_time_distribution.csv', index=False, encoding='cp949')
df_place_final.to_csv(output_dir + 'crime_place_distribution.csv', index=False, encoding='cp949')

print("\n저장된 파일:")
print(f"   1. crime_yearly_2014_2024.csv (시계열 데이터)")
print(f"   2. crime_time_distribution.csv (시간대별 분포)")
print(f"   3. crime_place_distribution.csv (장소별 분포)")

print("\n" + "="*80)
print("✅ 전처리 완료! 머신러닝 학습 준비 완료")
print("="*80)

📊 서울시 5대 범죄 데이터 전처리 - 머신러닝용

[PART 1] 시계열 데이터 재구조화 (2014-2024)
--------------------------------------------------------------------------------
✅ 파일 로드 성공 (인코딩: utf-8)

원본 데이터 형태: (29, 134)
데이터 행 수: 26

컬럼 정보 샘플:
  original_col  year crime_type data_type
0         2014  2014         소계        발생
1       2014.1  2014         소계        검거
2       2014.2  2014         살인        발생
3       2014.3  2014         살인        검거
4       2014.4  2014         강도        발생
5       2014.5  2014         강도        검거
6       2014.6  2014    강간·강제추행        발생
7       2014.7  2014    강간·강제추행        검거
8       2014.8  2014         절도        발생
9       2014.9  2014         절도        검거

'발생' 데이터 컬럼 수: 55

자치구 수: 25
자치구 목록: ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']
✅ 시계열 데이터 재구조화 완료
   - 최종 행 수: 1,375개
   - 연도 범위: 2014 ~ 2024
   - 자치구 수: 25개
   - 범죄 종류: 강도, 살인, 성폭력, 절도, 폭력

In [4]:
"""
서울시 범죄 데이터 결측치 확인 스크립트
- 원본 데이터의 결측치 현황
- 전처리 후 데이터의 결측치 현황
- 시각화를 통한 결측치 패턴 분석
"""

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# ============================================
# 1. 데이터 로드 함수
# ============================================

def load_csv_with_encoding(file_path):
    """인코딩 예외 처리"""
    encodings = ['cp949', 'utf-8', 'euc-kr']
    
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            return df
        except:
            continue
    raise ValueError(f"❌ '{file_path}' 파일을 읽을 수 없습니다.")


print("="*80)
print("🔍 서울시 범죄 데이터 결측치 확인")
print("="*80)


# ============================================
# 2. 원본 데이터 결측치 확인
# ============================================

print("\n[1] 원본 데이터 결측치 현황")
print("-"*80)

# 시간 데이터
df_time_original = load_csv_with_encoding('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/csv/범죄발생시간.csv')
print("\n📊 [범죄발생시간.csv]")
print(f"   데이터 형태: {df_time_original.shape}")

# 표준 결측치 (NaN) 확인
null_count_time = df_time_original.isnull().sum()
print(f"\n   1) 표준 결측치 (NaN) 개수:")
if null_count_time.sum() == 0:
    print("      → 표준 결측치 없음 ✅")
else:
    print(null_count_time[null_count_time > 0])

# '-' 문자 결측치 확인
print(f"\n   2) '-' 문자로 표현된 결측치:")
dash_counts_time = {}
for col in df_time_original.columns:
    if col != '범죄분류':
        dash_count = (df_time_original[col].astype(str) == '-').sum()
        if dash_count > 0:
            dash_counts_time[col] = dash_count

if dash_counts_time:
    for col, count in dash_counts_time.items():
        print(f"      {col}: {count}개")
else:
    print("      → '-' 결측치 없음 ✅")

# 빈 문자열 결측치 확인
print(f"\n   3) 빈 문자열('') 결측치:")
empty_counts_time = {}
for col in df_time_original.columns:
    if col != '범죄분류':
        empty_count = (df_time_original[col].astype(str).str.strip() == '').sum()
        if empty_count > 0:
            empty_counts_time[col] = empty_count

if empty_counts_time:
    for col, count in empty_counts_time.items():
        print(f"      {col}: {count}개")
else:
    print("      → 빈 문자열 결측치 없음 ✅")


# 장소 데이터
df_place_original = load_csv_with_encoding('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/csv/범죄발생장소.csv')
print("\n\n📊 [범죄발생장소.csv]")
print(f"   데이터 형태: {df_place_original.shape}")

# 표준 결측치 (NaN) 확인
null_count_place = df_place_original.isnull().sum()
print(f"\n   1) 표준 결측치 (NaN) 개수:")
if null_count_place.sum() == 0:
    print("      → 표준 결측치 없음 ✅")
else:
    print(null_count_place[null_count_place > 0])

# '-' 문자 결측치 확인
print(f"\n   2) '-' 문자로 표현된 결측치:")
dash_counts_place = {}
for col in df_place_original.columns:
    if col != '범죄분류':
        dash_count = (df_place_original[col].astype(str) == '-').sum()
        if dash_count > 0:
            dash_counts_place[col] = dash_count

if dash_counts_place:
    total_dash = sum(dash_counts_place.values())
    print(f"      → 총 {total_dash}개의 '-' 결측치 발견")
    print(f"      → 영향받는 컬럼 수: {len(dash_counts_place)}개")
    print("\n      상위 5개 컬럼:")
    sorted_dash = sorted(dash_counts_place.items(), key=lambda x: x[1], reverse=True)
    for col, count in sorted_dash[:5]:
        print(f"        • {col}: {count}개")
else:
    print("      → '-' 결측치 없음 ✅")

# 빈 문자열 결측치 확인
print(f"\n   3) 빈 문자열('') 결측치:")
empty_counts_place = {}
for col in df_place_original.columns:
    if col != '범죄분류':
        empty_count = (df_place_original[col].astype(str).str.strip() == '').sum()
        if empty_count > 0:
            empty_counts_place[col] = empty_count

if empty_counts_place:
    for col, count in empty_counts_place.items():
        print(f"      {col}: {count}개")
else:
    print("      → 빈 문자열 결측치 없음 ✅")


# ============================================
# 3. 전처리 후 데이터 결측치 확인
# ============================================

print("\n\n" + "="*80)
print("[2] 전처리 후 데이터 결측치 현황")
print("-"*80)

# 전처리된 데이터 로드
df_time_processed = pd.read_csv('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/범죄발생시간_전처리.csv', encoding='cp949')
df_place_processed = pd.read_csv('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/범죄발생장소_전처리.csv', encoding='cp949')

print("\n📊 [범죄발생시간_전처리.csv]")
print(f"   데이터 형태: {df_time_processed.shape}")

null_time_processed = df_time_processed.isnull().sum()
print(f"\n   표준 결측치 (NaN) 개수:")
if null_time_processed.sum() == 0:
    print("      → 결측치 없음 ✅")
else:
    print(null_time_processed[null_time_processed > 0])

print("\n📊 [범죄발생장소_전처리.csv]")
print(f"   데이터 형태: {df_place_processed.shape}")

null_place_processed = df_place_processed.isnull().sum()
print(f"\n   표준 결측치 (NaN) 개수:")
if null_place_processed.sum() == 0:
    print("      → 결측치 없음 ✅")
else:
    print(null_place_processed[null_place_processed > 0])


# ============================================
# 4. 결측치 처리 전후 비교
# ============================================

print("\n\n" + "="*80)
print("[3] 결측치 처리 전후 비교")
print("-"*80)

# 시간 데이터
print("\n📊 시간 데이터")
total_cells_time_orig = df_time_original.shape[0] * (df_time_original.shape[1] - 1)
total_dash_time = sum(dash_counts_time.values()) if dash_counts_time else 0
total_empty_time = sum(empty_counts_time.values()) if empty_counts_time else 0
total_missing_time = total_dash_time + total_empty_time

print(f"   원본 데이터:")
print(f"   - 전체 셀 개수: {total_cells_time_orig:,}개")
print(f"   - '-' 결측치: {total_dash_time}개")
print(f"   - 빈 문자열 결측치: {total_empty_time}개")
print(f"   - 총 결측치: {total_missing_time}개 ({total_missing_time/total_cells_time_orig*100:.2f}%)")

total_cells_time_proc = df_time_processed.shape[0] * (df_time_processed.shape[1] - 1)
total_missing_time_proc = null_time_processed.sum()

print(f"\n   전처리 후:")
print(f"   - 전체 셀 개수: {total_cells_time_proc:,}개")
print(f"   - 결측치: {total_missing_time_proc}개 (0.00%)")
print(f"   ✅ 결측치 처리 완료!")

# 장소 데이터
print("\n📊 장소 데이터")
total_cells_place_orig = df_place_original.shape[0] * (df_place_original.shape[1] - 1)
total_dash_place = sum(dash_counts_place.values()) if dash_counts_place else 0
total_empty_place = sum(empty_counts_place.values()) if empty_counts_place else 0
total_missing_place = total_dash_place + total_empty_place

print(f"   원본 데이터:")
print(f"   - 전체 셀 개수: {total_cells_place_orig:,}개")
print(f"   - '-' 결측치: {total_dash_place}개")
print(f"   - 빈 문자열 결측치: {total_empty_place}개")
print(f"   - 총 결측치: {total_missing_place}개 ({total_missing_place/total_cells_place_orig*100:.2f}%)")

total_cells_place_proc = df_place_processed.shape[0] * (df_place_processed.shape[1] - 1)
total_missing_place_proc = null_place_processed.sum()

print(f"\n   전처리 후:")
print(f"   - 전체 셀 개수: {total_cells_place_proc:,}개")
print(f"   - 결측치: {total_missing_place_proc}개 (0.00%)")
print(f"   ✅ 결측치 처리 완료!")


# ============================================
# 5. 상세 결측치 패턴 분석 (원본 데이터)
# ============================================

print("\n\n" + "="*80)
print("[4] 원본 데이터의 결측치 패턴 상세 분석")
print("-"*80)

# 시간 데이터 - 범죄별 결측치 확인
print("\n📊 시간 데이터 - 범죄 유형별 결측치")
print("-"*80)
for idx, row in df_time_original.iterrows():
    crime_type = row['범죄분류']
    dash_count = sum((row[col] == '-') for col in df_time_original.columns if col != '범죄분류')
    
    if dash_count > 0:
        print(f"   {crime_type}: {dash_count}개 컬럼에 '-' 존재")
        dash_cols = [col for col in df_time_original.columns if col != '범죄분류' and row[col] == '-']
        print(f"      → {', '.join(dash_cols[:3])}{'...' if len(dash_cols) > 3 else ''}")

# 장소 데이터 - 범죄별 결측치 확인
print("\n📊 장소 데이터 - 범죄 유형별 결측치")
print("-"*80)
for idx, row in df_place_original.iterrows():
    crime_type = row['범죄분류']
    dash_count = sum((str(row[col]) == '-') for col in df_place_original.columns if col != '범죄분류')
    
    if dash_count > 0:
        print(f"   {crime_type}: {dash_count}개 컬럼에 '-' 존재")


# ============================================
# 6. 결측치 처리 방법 요약
# ============================================

print("\n\n" + "="*80)
print("[5] 결측치 처리 방법 요약")
print("-"*80)

print("""
📌 적용된 결측치 처리 방법:

1. '-' 문자 → 0 변환
   - 범죄 발생이 없거나 집계되지 않은 경우를 0으로 처리
   - 통계 분석에 적합한 숫자형 데이터로 변환

2. 빈 문자열('') → 0 변환
   - 데이터 누락을 0으로 간주
   
3. 자료형 변환
   - 모든 수치 데이터를 int형으로 통일
   - 분석 및 계산의 일관성 확보

4. 검증
   - 전처리 후 NaN 결측치 없음을 확인
   - 모든 수치 데이터가 정상적으로 변환되었는지 확인
""")

print("="*80)
print("✅ 결측치 확인 완료!")
print("="*80)

🔍 서울시 범죄 데이터 결측치 확인

[1] 원본 데이터 결측치 현황
--------------------------------------------------------------------------------

📊 [범죄발생시간.csv]
   데이터 형태: (31, 10)

   1) 표준 결측치 (NaN) 개수:
      → 표준 결측치 없음 ✅

   2) '-' 문자로 표현된 결측치:
      → '-' 결측치 없음 ✅

   3) 빈 문자열('') 결측치:
      → 빈 문자열 결측치 없음 ✅


📊 [범죄발생장소.csv]
   데이터 형태: (28, 28)

   1) 표준 결측치 (NaN) 개수:
      → 표준 결측치 없음 ✅

   2) '-' 문자로 표현된 결측치:
      → '-' 결측치 없음 ✅

   3) 빈 문자열('') 결측치:
      → 빈 문자열 결측치 없음 ✅


[2] 전처리 후 데이터 결측치 현황
--------------------------------------------------------------------------------

📊 [범죄발생시간_전처리.csv]
   데이터 형태: (5, 10)

   표준 결측치 (NaN) 개수:
      → 결측치 없음 ✅

📊 [범죄발생장소_전처리.csv]
   데이터 형태: (5, 5)

   표준 결측치 (NaN) 개수:
      → 결측치 없음 ✅


[3] 결측치 처리 전후 비교
--------------------------------------------------------------------------------

📊 시간 데이터
   원본 데이터:
   - 전체 셀 개수: 279개
   - '-' 결측치: 0개
   - 빈 문자열 결측치: 0개
   - 총 결측치: 0개 (0.00%)

   전처리 후:
   - 전체 셀 개수: 45개
   - 결측치: 0개 (0.00%)
   ✅ 결측치 처리 완료!

📊 장소 데이터
   원본 데